In [1]:
import pandas as pd

# 업로드된 히트 단위 데이터 로딩
df = pd.read_csv("data.csv")

# 기본적으로 필요한 컬럼이 있는지 확인
df.columns


Index(['fullVisitorId', 'visitorId', 'visitId', 'date', 'visitStartTime',
       'channelGrouping', 'visitNumber', 'hits', 'pageviews', 'timeOnSite',
       'newVisits', 'transactionRevenue', 'transactions', 'bounces',
       'totalTransactionRevenue', 'campaign', 'referralPath', 'medium',
       'browser', 'deviceCategory', 'isMobile', 'operatingSystem', 'continent',
       'longitude', 'latitude', 'city', 'region', 'country', 'subContinent',
       'hitNumber', 'referer', 'isExit', 'isEntrance', 'isInteraction',
       'minute', 'hour', 'time', 'type', 'dataSource', 'pagePath', 'pageTitle',
       'screenName', 'exitScreenName', 'landingScreenName', 'isFatal',
       'eventAction', 'eventLabel', 'v2ProductName', 'productListPosition',
       'productListName', 'isClick', 'isImpression', 'productQuantity',
       'productPrice', 'productRevenue', 'v2ProductCategory', 'promoId',
       'promoPosition', 'promoCreative', 'promoName', 'promoIsClick',
       'action_type', 'step', 'socialN

In [2]:
# 세션 단위
df["session"] = (
    df["fullVisitorId"].astype(str)
    + "_" + df["visitId"].astype(str)
)

# 전환 여부
df["converted"] = df["transactions"].fillna(0).apply(lambda x: 1 if x >= 1 else 0)

# 세션 내 페이지 방문 경로(pagePathFlow)

-  세션 단위([fullVisitorId, visitStartTime]): session
- 전환 여부(transactions >= 1): converted
- 페이지 경로 탐색 깊이: steps
-  \> 로 페이지 경로 연결



In [3]:
import pandas as pd

# time 컬럼이 정수형이 아닐 수 있으므로 안전하게 변환
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# 세션 + 시간 순 정렬
df_sorted = df.sort_values(by=["session", "time"])

# 세션별 페이지 흐름 생성
page_path_flow = (
    df_sorted.groupby("session")["pagePath"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="pagePathFlow")
)

# 원본 df에 병합
df = df.merge(page_path_flow, on="session", how="left")

# 결과 확인
df[["session", "pagePathFlow","hits"]].drop_duplicates().head()


,session,pagePathFlow
0,3950961448949182556_1487008581,/home-2 > /home-2 > /home-2 > /home-2 > /home-2


# 히트별 시작 시간(hitTimeFlow)

In [5]:
import pandas as pd

# 예시: df에 hour, minute 컬럼이 존재한다고 가정
# 두 컬럼을 이용해서 'HH:MM' 형식의 문자열 생성
df["hit_time"] = df["hour"].astype(int).astype(str).str.zfill(2) + ":" + df["minute"].astype(int).astype(str).str.zfill(2)

# 결과 확인
df[["hour", "minute", "hit_time"]].head()


,hour,minute,hit_time
0,9,56,09:56
1,9,57,09:57
2,9,57,09:57
3,9,57,09:57
4,9,57,09:57


In [6]:
# 세션 기준, 시간 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# 세션별 hit_time 흐름 생성
hit_time_flow = (
    df_sorted.groupby("session")["hit_time"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="hitTimeFlow")
)

# 원본 df에 병합
df = df.merge(hit_time_flow, on="session", how="left")

# 결과 확인
df[["session", "hitTimeFlow"]].drop_duplicates().head()


,session,hitTimeFlow
0,3950961448949182556_1487008581,09:56 > 09:57 > 09:57 > 09:57 > 09:57


# 히트별 체류 시간(hit_time_spent)

In [ ]:
# step 1: 세션 내 히트 정렬
df = df.sort_values(by=["session", "time"])

# step 2: 다음 히트의 시간 차이 → 체류 시간 (ms 단위 → 초 단위 변환)
df["next_time"] = df.groupby("session")["time"].shift(-1)
df["hit_time_spent"] = (df["next_time"] - df["time"]) / 1000  # ms → sec
df["hit_time_spent"] = df["hit_time_spent"].fillna(0).round(2)

# step 3: 세션별 히트 체류 시간 경로 생성 (초 단위)
hit_time_path = (
    df.groupby("session")["hit_time_spent"]
    .apply(lambda x: " > ".join(x.astype(str)))
    .reset_index(name="hitTimeSpentPath")
)

# step 4: 원본 df에 병합
df = df.merge(hit_time_path, on="session", how="left")

# 결과 예시 출력
df[["session", "hitTimeSpentPath"]].drop_duplicates().head()

,session,hitTimeSpentPath
0,1000023789490173030_1479015374,0.0
1,100004954768738210_1478493459,0.0
2,1000168899660186477_1496866124,0.0
3,1000545479458238020_1470939857,0.0
4,1001530906644524820_1494569474,0.0


> 0.0초 체류시간
-  같은 페이지 또는 동일한 타임스탬프 반복 기록
- 의미 없는 반복 클릭 또는 자동 이벤트
- 마지막 히트는 항상 0


# 사용자 상호작용 유형 흐름(typeFlow)

In [ ]:
import pandas as pd

# 세션 기준, 히트 순서 기준 정렬
df_sorted = df.sort_values(by=["session", "hitNumber"])

# 세션별 type 흐름 생성
type_flow = (
    df_sorted.groupby("session")["type"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="typeFlow")
)

# 원본 df에 병합
df = df.merge(type_flow, on="session", how="left")

# 결과 확인
df[["session", "typeFlow"]].drop_duplicates().head()


,session,typeFlow
0,1000023789490173030_1479015374,PAGE
1,100004954768738210_1478493459,PAGE
2,1000168899660186477_1496866124,PAGE
3,1000545479458238020_1470939857,PAGE
4,1001530906644524820_1494569474,PAGE


# 세션 내 방문 페이지 제목 흐름(pageTitlePath)

In [ ]:
df[['pageTitle','fullVisitorId']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pageTitle      49995 non-null  object
 1   fullVisitorId  50000 non-null  uint64
dtypes: object(1), uint64(1)
memory usage: 781.4+ KB


In [ ]:
# session 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# session별 pageTitle 경로 생성
page_title_path = (
    df_sorted.groupby("session")["pageTitle"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)) if x.dropna().shape[0] > 0 else pd.NA)
    .reset_index(name="pageTitlePath")
)

# 병합: session 기준으로 기존 df에 추가
df = df.merge(page_title_path, on="session", how="left")

# 결과 확인: pageTitlePath가 존재하는 일부 샘플만 보기
df[df["pageTitlePath"].notna()][["session", "pageTitlePath"]].head()


,session,pageTitlePath
0,1000023789490173030_1479015374,Store search results
1,100004954768738210_1478493459,More Bags | Bags | Google Merchandise Store
2,1000168899660186477_1496866124,Home
3,1000545479458238020_1470939857,Drinkware
4,1001530906644524820_1494569474,Shop by Brand | Google Merchandise Store


In [ ]:
df[['fullVisitorId','pageTitlePath']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   fullVisitorId  50000 non-null  uint64
 1   pageTitlePath  49995 non-null  object
dtypes: object(1), uint64(1)
memory usage: 781.4+ KB


# 세션 내 주요 이벤트 흐름(eventActionPath)

In [ ]:
df[['eventAction','fullVisitorId']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   eventAction    1273 non-null   object
 1   fullVisitorId  50000 non-null  uint64
dtypes: object(1), uint64(1)
memory usage: 781.4+ KB


In [ ]:
import pandas as pd

# 'time' 컬럼 숫자형으로 변환
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# session 기준으로 정렬
df_sorted = df.sort_values(by=["session", "time"])

# session별 eventActionPath 생성
event_action_path = (
    df_sorted.groupby("session")["eventAction"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)) if x.dropna().shape[0] > 0 else pd.NA)
    .reset_index(name="eventActionPath")
)

# 확인: 비어있지 않은 경로만 존재하는지 먼저 체크
print("eventActionPath 생성된 수:", len(event_action_path))

# 병합: session 기준으로 기존 df에 추가
df = df.merge(event_action_path, on="session", how="left")

# 결과 확인: eventActionPath가 존재하는 일부 샘플만 보기
df[df["eventActionPath"].notna()][["session", "eventActionPath"]].head()

eventActionPath 생성된 수: 44060


,session,eventActionPath
66,1012233368984059196_1493409446,Add to Cart
123,1019181007381380154_1474854553,Quickview Click
147,102355785728047008_1476117074,Quickview Click
181,1030000529728750128_1481912723,Quickview Click
283,1050808830592816081_1471361358,Quickview Click


In [ ]:
df[["session", "eventActionPath"]].head()

,session,eventActionPath
0,76529729049367813_1482366901_20161221,Quickview Click > Add to Cart
1,76529729049367813_1482366901_20161221,Quickview Click > Add to Cart
2,76529729049367813_1482366901_20161221,Quickview Click > Add to Cart
3,76529729049367813_1482366901_20161221,Quickview Click > Add to Cart
4,76529729049367813_1482366901_20161221,Quickview Click > Add to Cart


In [ ]:
df[['fullVisitorId','eventActionPath']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fullVisitorId    50000 non-null  uint64
 1   eventActionPath  2256 non-null   object
dtypes: object(1), uint64(1)
memory usage: 781.4+ KB


# 세션 내 이벤트 분석 흐름(eventLabelPath)

In [ ]:
import pandas as pd

# 'time' 컬럼 숫자형으로 변환 (안전성 확보)
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# 세션 기준 + 시간 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# session별 eventLabelPath 생성
event_label_path = (
    df_sorted.groupby("session")["eventLabel"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)) if x.dropna().shape[0] > 0 else pd.NA)
    .reset_index(name="eventLabelPath")
)

# 원본 df에 병합
df = df.merge(event_label_path, on="session", how="left")

# 결과 확인: 값이 존재하는 경우만 출력
df[df["eventLabelPath"].notna()][["session", "eventLabelPath"]].drop_duplicates().head()


,session,eventLabelPath
123,1019181007381380154_1474854553,YouTube Wool Heather Cap Heather/Black
147,102355785728047008_1476117074,Google Rucksack
181,1030000529728750128_1481912723,Google Women's Lightweight Microfleece Jacket
283,1050808830592816081_1471361358,8 pc Android Sticker Sheet
309,1057078399235259309_1473999562,Google Alpine Style Backpack


# product 필드 (productSummaryPath)

> productSummary

1. 상품 이름
2. 리스트 내 위치
3. 리스트 이름
4. 클릭 여부
5. 노출 여부
6. 구매 수량
7. 상품 개당 가격
8. 상품별 발생 수익
9. 상품 카테고리

In [ ]:
import pandas as pd

# 정수값은 소수점 없이 출력하도록 처리
def format_value(val):
    if pd.isnull(val):
        return "N/A"
    if isinstance(val, (int, float)) and val == int(val):
        return str(int(val))
    return str(val)

# 번호와 함께 각 컬럼을 정리
def create_product_summary(row):
    return " | ".join([
        f"1.{format_value(row.get('v2ProductName'))}",
        f"2.{format_value(row.get('productListPosition'))}",
        f"3.{format_value(row.get('productListName'))}",
        f"4.{format_value(row.get('isClick'))}",
        f"5.{format_value(row.get('isImpression'))}",
        f"6.{format_value(row.get('productQuantity'))}",
        f"7.{format_value(row.get('productPrice'))}",
        f"8.{format_value(row.get('productRevenue'))}",
        f"9.{format_value(row.get('v2ProductCategory'))}",
    ])

# 새로운 컬럼 생성
df["productSummary"] = df.apply(create_product_summary, axis=1)

# 결과 확인
df[["productSummary"]].head()


,productSummary
0,1.Google Women's Long Sleeve Tee Lavender | 2....
1,1.Suitcase Organizer Cubes | 2.2 | 3.Category ...
2,1.N/A | 2.N/A | 3.N/A | 4.N/A | 5.N/A | 6.N/A ...
3,1.Red Shine 15 oz Mug | 2.13 | 3.Category | 4....
4,1.Windup Android | 2.5 | 3.Category | 4.N/A | ...


In [ ]:
import pandas as pd

# 'time' 컬럼을 숫자로 변환 (정렬을 위해 필요)
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# 세션 기준 + 시간 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# 세션별 promotionSummary 흐름 생성 (모든 값 포함)
promotion_summary_path = (
    df_sorted.groupby("session")["productSummary"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="productSummaryPath")
)

# 원본 df에 병합
df = df.merge(promotion_summary_path, on="session", how="left")

# 결과 확인
df[["session", "productSummaryPath"]].drop_duplicates().head()


,session,productSummaryPath
0,1000023789490173030_1479015374,1.Google Women's Long Sleeve Tee Lavender | 2....
1,100004954768738210_1478493459,1.Suitcase Organizer Cubes | 2.2 | 3.Category ...
2,1000168899660186477_1496866124,1.N/A | 2.N/A | 3.N/A | 4.N/A | 5.N/A | 6.N/A ...
3,1000545479458238020_1470939857,1.Red Shine 15 oz Mug | 2.13 | 3.Category | 4....
4,1001530906644524820_1494569474,1.Windup Android | 2.5 | 3.Category | 4.N/A | ...


# promotionSummaryPath


> promotionSummary

1. 프로모션 클릭 여부
2. 프로모션 이름
3. 프로모션 위치
4. 프로모션에서 사용된 배너 및 이미지

In [ ]:
import pandas as pd

# 정수값은 소수점 없이 출력하도록 처리
def format_value(val):
    if pd.isnull(val):
        return "N/A"
    if isinstance(val, (int, float)) and val == int(val):
        return str(int(val))
    return str(val)

# 번호와 함께 각 컬럼을 정리
def create_product_summary(row):
    return " | ".join([
        f"1.{format_value(row.get('promoIsClick'))}",
        f"2.{format_value(row.get('promoName'))}",
        f"3.{format_value(row.get('promoPosition'))}",
        f"4.{format_value(row.get('promoCreative'))}",
    ])

# 새로운 컬럼 생성
df["promotionSummary"] = df.apply(create_product_summary, axis=1)

# 결과 확인
df[["promotionSummary"]].head()


,promotionSummary
0,1.N/A | 2.N/A | 3.N/A | 4.N/A
1,1.N/A | 2.N/A | 3.N/A | 4.N/A
2,1.N/A | 2.Womens T-Shirts | 3.Row 3-2 | 4.wome...
3,1.N/A | 2.N/A | 3.N/A | 4.N/A
4,1.N/A | 2.N/A | 3.N/A | 4.N/A


In [ ]:
# promotionSummary 값이 모두 "N/A"로 구성된 행 제외
df_valid_promo = df[df["promotionSummary"] != "1.N/A | 2.N/A | 3.N/A | 4.N/A"]

# 결과 확인
df_valid_promo[["promotionSummary"]].head(10)


,promotionSummary
2,1.N/A | 2.Womens T-Shirts | 3.Row 3-2 | 4.wome...
5,1.N/A | 2.Apparel | 3.Row 1 | 4.home_main_link...
6,1.N/A | 2.Office | 3.Row 5 Color Combo | 4.gre...
9,1.N/A | 2.Office | 3.Row 5 Color Combo | 4.gre...
12,1.N/A | 2.Backpacks | 3.Row 2 Combo | 4.home_b...
17,1.N/A | 2.Apparel | 3.Row 1 | 4.home_main_link...
18,1.N/A | 2.YouTube Brand | 3.Brand Row 7-2 | 4....
21,1.N/A | 2.Womens T-Shirts | 3.Row 3-2 | 4.wome...
22,1.N/A | 2.Backpacks | 3.Row 2 Combo | 4.home_b...
26,1.N/A | 2.YouTube Brand | 3.Brand Row 7-2 | 4....


In [ ]:
import pandas as pd

# 'time' 컬럼을 숫자로 변환 (정렬을 위해 필요)
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# 세션 기준 + 시간 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# 세션별 promotionSummary 흐름 생성 (모든 값 포함)
promotion_summary_path = (
    df_sorted.groupby("session")["promotionSummary"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="promotionSummaryPath")
)

# 원본 df에 병합
df = df.merge(promotion_summary_path, on="session", how="left")

# 결과 확인
df[["session", "promotionSummaryPath"]].drop_duplicates().head()


,session,promotionSummaryPath
0,1000023789490173030_1479015374,1.N/A | 2.N/A | 3.N/A | 4.N/A
1,100004954768738210_1478493459,1.N/A | 2.N/A | 3.N/A | 4.N/A
2,1000168899660186477_1496866124,1.N/A | 2.Womens T-Shirts | 3.Row 3-2 | 4.wome...
3,1000545479458238020_1470939857,1.N/A | 2.N/A | 3.N/A | 4.N/A
4,1001530906644524820_1494569474,1.N/A | 2.N/A | 3.N/A | 4.N/A


# 세션 기준 전자상거래 행동 흐름(actionTypePath)

In [ ]:
df[["session","action_type"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   session      50000 non-null  object
 1   action_type  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [ ]:
# step 반영 action type
def convert_action_with_step(row):
    if row["action_type"] == 5 and pd.notnull(row["step"]):
        return f"4.{int(row['step'])}"
    elif pd.notnull(row["action_type"]):
        return str(int(row["action_type"]))
    else:
        return pd.NA

df["action_type"] = df.apply(convert_action_with_step, axis=1)


ValueError: invalid literal for int() with base 10: '4.1'

In [ ]:
import pandas as pd

# 'time' 컬럼 숫자형으로 변환 (안정성 확보)
df["time"] = pd.to_numeric(df["time"], errors="coerce")

# session + time 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# ⚠️ astype(int) 제거하고 문자열(str)로 처리
# 1. 전체 action_type 흐름 (0 포함)
action_type_path = (
    df_sorted.groupby("session")["action_type"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)) if x.dropna().shape[0] > 0 else pd.NA)
    .reset_index(name="actionTypePath")
)

# 2. action_type == '0' 또는 0 제거 후 흐름
# → 숫자일 수도, 문자열일 수도 있기 때문에 양쪽 대응
df_filtered = df_sorted[~df_sorted["action_type"].isin([0, '0'])]

action_type_path_remove = (
    df_filtered.groupby("session")["action_type"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)) if x.dropna().shape[0] > 0 else pd.NA)
    .reset_index(name="actionTypePathRemove")
)

# 3. 병합
df = df.merge(action_type_path, on="session", how="left")
df = df.merge(action_type_path_remove, on="session", how="left")

# 결과 확인
df[["session", "actionTypePath", "actionTypePathRemove"]].drop_duplicates().head()


,session,actionTypePath,actionTypePathRemove
0,1000023789490173030_1479015374,0,NaN
1,100004954768738210_1478493459,0,NaN
2,1000168899660186477_1496866124,0,NaN
3,1000545479458238020_1470939857,0,NaN
4,1001530906644524820_1494569474,0,NaN


# 세션 기준 전자상거래 최대 깊이 행동(maxActionType)

In [ ]:
import pandas as pd

# 1. action_type이 숫자인지 확인하고, 안전하게 변환
df["action_type"] = pd.to_numeric(df["action_type"], errors="coerce")

# 2. 세션 단위 최대 퍼널 스텝 계산 (action_type의 최대값)
max_action_type = (
    df.groupby("session")["action_type"]
    .max()
    .reset_index(name="maxActionType")
)

# 3. 원본 df에 병합
df = df.merge(max_action_type, on="session", how="left")

# 4. 결과 확인
df[["session", "action_type", "maxActionType"]].head()


,session,action_type,maxActionType
0,1000023789490173030_1479015374,0.0,0.0
1,100004954768738210_1478493459,0.0,0.0
2,1000168899660186477_1496866124,0.0,0.0
3,1000545479458238020_1470939857,0.0,0.0
4,1001530906644524820_1494569474,0.0,0.0


# 유입 소셜 네트워크 이름(socialNetwork)

In [ ]:
# 세션 기준으로 socialNetwork의 유니크한 개수 계산
session_social_counts = (
    df.groupby("session")["socialNetwork"]
    .nunique()
    .reset_index(name="unique_social_count")
)

# 유니크한 값이 2개 이상인 세션만 확인
multiple_social_sessions = session_social_counts[session_social_counts["unique_social_count"] > 1]

# 결과 확인
print(f"⚠️ 서로 다른 socialNetwork가 존재하는 세션 수: {len(multiple_social_sessions)}")
multiple_social_sessions.head()


⚠️ 서로 다른 socialNetwork가 존재하는 세션 수: 0


,session,unique_social_count


In [ ]:
import pandas as pd

# 세션 단위로 대표 socialNetwork 추출
social_by_session = (
    df.groupby("session")["socialNetwork"]
    .first()  # 모든 세션에 하나의 socialNetwork만 존재한다고 확인됨
    .reset_index(name="socialNetwork_session")
)

# 기존 df에 병합
df = df.merge(social_by_session, on="session", how="left")

# 결과 확인
df[["session", "socialNetwork", "socialNetwork_session"]].drop_duplicates().head()


,session,socialNetwork,socialNetwork_session
0,1000023789490173030_1479015374,(not set),(not set)
1,100004954768738210_1478493459,(not set),(not set)
2,1000168899660186477_1496866124,(not set),(not set)
3,1000545479458238020_1470939857,(not set),(not set)
4,1001530906644524820_1494569474,Twitter,Twitter


# 콘텐츠 카테고리 흐름(contentGroupPath)

In [ ]:
import pandas as pd

# 콘텐츠 그룹 컬럼 리스트 정의
content_group_cols = ["contentGroup1", "contentGroup2", "contentGroup3"]

# contentGroup_combined 컬럼 생성 (빈값 제외하고 /로 연결)
df["contentGroup_combined"] = df[content_group_cols].fillna("").agg(" / ".join, axis=1)
df["contentGroup_combined"] = df["contentGroup_combined"].str.replace(r"( / )+", " / ", regex=True)  # 중복 구분자 제거
df["contentGroup_combined"] = df["contentGroup_combined"].str.strip(" /")  # 앞뒤 구분자 제거

# 세션 + 시간 기준 정렬
df_sorted = df.sort_values(by=["session", "time"])

# 세션별 콘텐츠 흐름 생성
content_group_path = (
    df_sorted.groupby("session")["contentGroup_combined"]
    .apply(lambda x: " > ".join(x.dropna().astype(str)))
    .reset_index(name="contentGroupPath")
)

# 원본 df에 병합
df = df.merge(content_group_path, on="session", how="left")

# 결과 확인 (중복 제거)
df[["session", "contentGroupPath"]].drop_duplicates().head()


,session,contentGroupPath
0,1000023789490173030_1479015374,(not set) / (not set) / (not set)
1,100004954768738210_1478493459,(not set) / Bags / (not set)
2,1000168899660186477_1496866124,(not set) / (not set) / (not set)
3,1000545479458238020_1470939857,(not set) / Drinkware / (not set)
4,1001530906644524820_1494569474,(not set) / Brands / (not set)


# 상호작용 히트 수(interactionHits) - 전체 true값으로 포함x


- 해당 히트가 사용자와의 의미 있는 상호작용인지 여부

In [ ]:
import pandas as pd

# 상호작용 히트 수 계산
interaction_counts = (
    df[df["isInteraction"] == True]  # 상호작용인 히트만 선택
    .groupby("session")["isInteraction"]
    .count()
    .reset_index(name="interactionCount")  # 세션별 상호작용 히트 수
)

# 원본 데이터에 병합 (session 기준)
df = df.merge(interaction_counts, on="session", how="left")

# 상호작용 히트가 없는 세션은 0으로 채움
df["interactionCount"] = df["interactionCount"].fillna(0).astype(int)

# 결과 예시 확인
df[["session", "interactionCount","steps"]].drop_duplicates().head()

,session,interactionCount,steps
0,1000023789490173030_1,1,1
1,1000545479458238020_1,1,1
2,1000810440227713794_1,1,1
3,1000816037989681455_2,1,1
4,1000816037989681455_4,2,2


In [ ]:
# step 1: 세션 기준으로 시간 순 정렬
df_sorted = df.sort_values(by=["session", "time"])

# step 2: isInteraction 경로 생성
interaction_path = (
    df_sorted.groupby("session")["isInteraction"]
    .apply(lambda x: " > ".join(x.fillna(False).astype(bool).astype(str)))
    .reset_index(name="interactionPath")
)

# step 3: 원본 df에 병합
df = df.merge(interaction_path, on="session", how="left")

# 결과 예시 확인
df[["session", "interactionPath"]].drop_duplicates().head()

,session,interactionPath
0,1000023789490173030_1,True
1,1000545479458238020_1,True
2,1000810440227713794_1,True
3,1000816037989681455_2,True
4,1000816037989681455_4,True > True


# 데이터 비교

In [ ]:
import pandas as pd

# (1) 전체 히트 수
total_hits = len(df)

# (2) exceptionInfo가 존재하는 히트 수 (isFatal 값이 결측치가 아닌 경우)
exception_hits = df["isFatal"].notna().sum()

# (3) 비율 계산
exception_ratio = (exception_hits / total_hits) * 100

# 결과 출력
print(f"전체 히트 수: {total_hits:,}")
print(f"exceptionInfo 존재 히트 수: {exception_hits:,}")
print(f"전체 대비 exceptionInfo 비율: {exception_ratio:.4f}%")


전체 히트 수: 50,000
exceptionInfo 존재 히트 수: 50,000
전체 대비 exceptionInfo 비율: 100.0000%


## promoId & promoPosition 비교

In [ ]:
df[["promoId","promoPosition"]].drop_duplicates().head(10)

,promoId,promoPosition
0,NaN,NaN
19,Apparel Row 1,Row 1
20,Backpacks Row 2 Combo,Row 2 Combo
21,Mens T-Shirts Row 3-1,Row 3-1
22,Womens T-Shirts Row 3-2,Row 3-2
23,Office Row 5 Color Combo,Row 5 Color Combo
24,Drinkware Row 4 Color Combo,Row 4 Color Combo
25,Google Brand Row 7-1,Brand Row 7-1
26,YouTube Brand Row 7-2,Brand Row 7-2
27,Android Brand Row 7-3,Brand Row 7-3


In [ ]:
# promoPosition이 NaN인 경우 False로 처리하고, 나머지는 포함 여부 확인
df["position_in_id"] = df.apply(
    lambda row: False if pd.isna(row["promoId"]) or pd.isna(row["promoPosition"])
    else str(row["promoPosition"]) in str(row["promoId"]),
    axis=1
)

# 포함 여부 통계 확인
print(df["position_in_id"].value_counts())

# 포함되는 일부 예시 출력
df[df["position_in_id"] == False][["promoId", "promoPosition"]].head()


position_in_id
False    38203
True     11797
Name: count, dtype: int64


,promoId,promoPosition
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [ ]:
# 포함되지 않는 경우 추출
df_not_included = df[df["position_in_id"] == False]

# promoId와 promoPosition이 모두 존재하는 행 중 일부 예시 출력
df_not_included[df_not_included["promoId"].notna() & df_not_included["promoPosition"].notna()][["promoId", "promoPosition"]].head()


,promoId,promoPosition


## promoId & promoName 비교

In [ ]:
df[["promoId","promoName","promoCreative"]].drop_duplicates().head(10)

,promoId,promoName,promoCreative
0,NaN,NaN,NaN
19,Apparel Row 1,Apparel,home_main_link_apparel.jpg
20,Backpacks Row 2 Combo,Backpacks,home_bags_google_2.jpg
21,Mens T-Shirts Row 3-1,Mens T-Shirts,mens-tshirts.jpg
22,Womens T-Shirts Row 3-2,Womens T-Shirts,womens-tshirts.jpg
23,Office Row 5 Color Combo,Office,green_row_link_to_office.jpg
24,Drinkware Row 4 Color Combo,Drinkware,red_row_hydrate.jpg
25,Google Brand Row 7-1,Google Brand,home_lower_google_500.jpg
26,YouTube Brand Row 7-2,YouTube Brand,home_lower_youtube_500.jpg
27,Android Brand Row 7-3,Andriod Brand,home_lower_android_500.jpg


In [ ]:
# promoPosition이 NaN인 경우 False로 처리하고, 나머지는 포함 여부 확인
df["Name_in_id"] = df.apply(
    lambda row: False if pd.isna(row["promoId"]) or pd.isna(row["promoName"])
    else str(row["promoName"]) in str(row["promoId"]),
    axis=1
)

# 포함 여부 통계 확인
print(df["Name_in_id"].value_counts())

# 포함되는 일부 예시 출력
df[df["Name_in_id"] == False][["promoId", "promoName"]].head()


Name_in_id
False    39444
True     10556
Name: count, dtype: int64


,promoId,promoName
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [ ]:
# 포함되지 않는 경우 추출
df_not_included = df[df["Name_in_id"] == False]

# promoId와 promoPosition이 모두 존재하는 행 중 일부 예시 출력
df_not_included[df_not_included["promoId"].notna() & df_not_included["promoName"].notna()][["promoId", "promoName"]].head()


,promoId,promoName
27,Android Brand Row 7-3,Andriod Brand
47,Android Brand Row 7-3,Andriod Brand
167,Android Brand Row 7-3,Andriod Brand
201,Android Brand Row 7-3,Andriod Brand
210,Android Brand Row 7-3,Andriod Brand


## pagePath & screenName 비교

In [ ]:
# 두 컬럼을 비교해서 일치하는지 여부를 나타내는 새로운 컬럼 생성
df["screen_equals_pagepath"] = df["screenName"] == df["pagePath"]

# 일치/불일치 건수 확인
print(df["screen_equals_pagepath"].value_counts())

# 일치하지 않는 데이터 샘플 몇 개 보기
df[df["screen_equals_pagepath"] == False][["screenName", "pagePath"]].drop_duplicates().head(10)


screen_equals_pagepath
False    50000
Name: count, dtype: int64


,screenName,pagePath
0,shop.googlemerchandisestore.com/google+redesign/apparel/men++s/men++s+outerwear/quickview,/google+redesign/apparel/men++s/men++s+outerwear/quickview
1,shop.googlemerchandisestore.com/signin.html,/signin.html
2,shop.googlemerchandisestore.com/yourinfo.html,/yourinfo.html
5,shop.googlemerchandisestore.com/payment.html,/payment.html
7,shop.googlemerchandisestore.com/revieworder.html,/revieworder.html
8,shop.googlemerchandisestore.com/ordercompleted.html,/ordercompleted.html
10,shop.googlemerchandisestore.com/google+redesign/apparel/quickview,/google+redesign/apparel/quickview
19,shop.googlemerchandisestore.com/home-2,/home-2
30,shop.googlemerchandisestore.com/registersuccess.html,/registersuccess.html
48,shop.googlemerchandisestore.com/google+redesign/apparel/kid+s,/google+redesign/apparel/kid+s


In [ ]:
# 1. screenName에서 'shop.googlemerchandisestore.com' 제거
df["screenName_cleaned"] = df["screenName"].str.replace("shop.googlemerchandisestore.com", "", regex=False)

# 2. 제거한 screenName과 pagePath 비교
df["screen_equals_pagepath_after_cleaning"] = df["screenName_cleaned"] == df["pagePath"]

# 3. 일치 여부 카운트 출력
print(df["screen_equals_pagepath_after_cleaning"].value_counts())

# 4. 여전히 불일치하는 경우 샘플 출력
df[df["screen_equals_pagepath_after_cleaning"] == False][["screenName", "screenName_cleaned", "pagePath"]].drop_duplicates().head(10)


screen_equals_pagepath_after_cleaning
True     49123
False      877
Name: count, dtype: int64


,screenName,screenName_cleaned,pagePath
528,www.googlemerchandisestore.com/google+redesign/brands/youtube/home,www.googlemerchandisestore.com/google+redesign/brands/youtube/home,/google+redesign/brands/youtube/home
49150,www.googlemerchandisestore.com/home,www.googlemerchandisestore.com/home,/home


In [ ]:
# 1. screenName에서 'shop.googlemerchandisestore.com'과 'www.googlemerchandisestore.com' 둘 다 제거
df["screenName_cleaned"] = df["screenName"].str.replace("shop.googlemerchandisestore.com", "", regex=False)
df["screenName_cleaned"] = df["screenName_cleaned"].str.replace("www.googlemerchandisestore.com", "", regex=False)

# 2. 제거한 screenName과 pagePath 비교
df["screen_equals_pagepath_after_cleaning"] = df["screenName_cleaned"] == df["pagePath"]

# 3. 일치 여부 카운트 출력
print(df["screen_equals_pagepath_after_cleaning"].value_counts())

# 4. 여전히 불일치하는 경우 샘플 출력
df[df["screen_equals_pagepath_after_cleaning"] == False][["screenName", "screenName_cleaned", "pagePath"]].drop_duplicates().head(10)


screen_equals_pagepath_after_cleaning
True    50000
Name: count, dtype: int64


,screenName,screenName_cleaned,pagePath
